In [ ]:

import os
from tqdm import tqdm
import pandas as pd
import torch
from torchvision.io import read_image, write_jpeg
from torch.utils.data import Dataset, DataLoader
from dataset import SingleImageDataset, FullLocationDataset

In [ ]:
def glue_images(dir_path, glued_path):
    glued = torch.cat([read_image(os.fsdecode(os.path.join(dir_path, path))) for path in os.listdir(dir_path)], dim=2)
    # print(glued.size())
    # print(glued_path)
    write_jpeg(glued, os.fsdecode(glued_path))

In [ ]:
data_dir = os.fsencode("/home/igosh/computer-vision-project/data/images")

annotations_list_unglued = []
annotations_list_glued = []

for location in tqdm(os.listdir(data_dir), total = 17600):
    if not os.path.isdir(data_dir + b"/" + location):
        continue
    location_dir_name: str = os.fsdecode(location)
    square_id, lat, lon = location_dir_name.split(",")
    for file in os.listdir(data_dir + b"/" + location):
        # print(os.fsdecode(file))
        if os.fsdecode(file) == "glued.jpg":
            continue
        angle = os.fsdecode(file)[:-4]
        path = location_dir_name + "/" + os.fsdecode(file)
        entry = [path, int(square_id), float(lat), float(lon), float(angle)]
        annotations_list_unglued.append(entry)
    glued_path = data_dir + b"/" + location + b"/glued.jpg"
    glue_images(data_dir + b"/" + location, glued_path)
    entry_glued = [location_dir_name + "/glued.jpg" , int(square_id), float(lat), float(lon)]
    annotations_list_glued.append(entry_glued)

annotations_unglued = pd.DataFrame(
    annotations_list_unglued, columns=["path", "square_id", "lat", "lon", "angle"]
).set_index("path")
annotations_unglued.to_csv(
    "/home/igosh/computer-vision-project/data/annotations_unglued.csv"
)
annotations_glued = pd.DataFrame(
    annotations_list_glued, columns=["path", "square_id", "lat", "lon"]
).set_index("path")
annotations_glued.to_csv(
    "/home/igosh/computer-vision-project/data/annotations_glued.csv"
)